In [12]:
!ls /kaggle/input/aksharantar/aksharantar_sampled/hin

hin_test.csv  hin_train.csv  hin_valid.csv


In [1]:
!pip install lightning wandb 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import wandb
from pytorch_lightning.loggers import WandbLogger

In [9]:
# common_path = "/kaggle/input/aksharantar/aksharantar_sampled/hin/"
common_path = "./aksharantar_sampled/hin/"
train_path = common_path +'hin_train.csv'
test_path = common_path +'hin_test.csv'
val_path = common_path +'hin_valid.csv'

In [10]:
train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)

In [11]:
train_input = train_data['shastragaar'].values
train_output = train_data['शस्त्रागार'].values

In [12]:
def get_dataloaders(path):
    data = pd.read_csv(path)
    data_input = data['shastragaar'].values
    data_output = data['शस्त्रागार'].values
    return data_input, data_output

In [45]:
def convert_word_to_tensor(word):
    char_to_int = {}
    char_to_int['SOW'] = 0
    char_to_int['EOW'] = 1
    count = 2
    for i in range(97, 123):
        char_to_int[chr(i)] = count 
        count += 1
    for i in range(2304, 2432):
        char_to_int[chr(i)] = count
        count += 1
    ascii = [char_to_int[c] for c in word]
    ascii.append(1) # for end of word
    one_hot = [torch.zeros(1, 1, 256) for _ in ascii]
    for i, c in enumerate(ascii):
        one_hot[i][0][0][c] = 1
    return one_hot

In [38]:
convert_word_to_tensor('शस्त्रागार')

[tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]),
 tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Encoder

In [40]:
class Encoder(pl.LightningModule):
    def __init__(self, cell_type, hidden_size, output_size, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout = dropout
        self.rnn = self.find_cell_type(cell_type)(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def find_cell_type(self, cell_type):
        if cell_type == "GRU":
            return nn.GRU
        elif cell_type == "LSTM":
            return nn.LSTM
        else:
            return nn.RNN

    def forward(self, input, hidden):
        output, hidden = self.rnn(input, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [41]:
enc = Encoder("GRU", 256, 128, 0.1)
enc

Encoder(
  (rnn): GRU(256, 256)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [46]:
Encoder.forward(enc, convert_word_to_tensor('lam')[0], enc.initHidden())

(tensor([[[ 0.0259,  0.0068,  0.0034,  0.0128, -0.0418,  0.0472, -0.0273,
           -0.0014, -0.0468, -0.0261,  0.0086,  0.0161,  0.0238,  0.0460,
           -0.0491,  0.0010, -0.0084, -0.0001, -0.0091, -0.0363, -0.0389,
           -0.0026,  0.0092,  0.0437, -0.0517,  0.0290,  0.0371,  0.0133,
           -0.0076, -0.0075, -0.0174,  0.0143,  0.0363, -0.0183,  0.0073,
           -0.0174,  0.0097, -0.0132, -0.0182,  0.0095,  0.0334, -0.0369,
           -0.0120, -0.0068, -0.0404, -0.0086, -0.0134, -0.0629,  0.0314,
            0.0300,  0.0113, -0.0198, -0.0060, -0.0119,  0.0358,  0.0338,
           -0.0173,  0.0304,  0.0101, -0.0003,  0.0368, -0.0167, -0.0553,
           -0.0039,  0.0057,  0.0070,  0.0302, -0.0244, -0.0040,  0.0280,
            0.0213, -0.0165,  0.0361,  0.0228, -0.0057,  0.0122, -0.0170,
            0.0224, -0.0073,  0.0186, -0.0407, -0.0350, -0.0228,  0.0279,
            0.0116, -0.0586, -0.0245,  0.0398, -0.0112, -0.0377, -0.0017,
           -0.0003, -0.0173,  0.0116, 

In [23]:
convert_word_to_tensor('शस्त्रागार')[0]

tensor(2358)